To use this notebook interactively, install the `bash_kernel` Python package with either:
* `pip install bash_kernel`
* `conda install -c conda-forge bash_kernel`

# OSS Module 01 - Git Basics

# Section 1: What is version control?

# Section 2: Git and GitHub setup

# Section 3:  A Git project directory

## Parts of a local Git project

When you download a Git repo, the contents are stored on your computer as normal files in a normal directory.  In Git terminology, these files are the **working tree**.  You can edit these files freely.  

On your computer, changes to the files are stored in the **local repository** or local **database**.  Soon, we will learn how to store (or **commit**) new versions of the files; and how to retrieve (or **checkout**) previous versions of the files.  

The contents of this repo (`oss-training`) are shown below.  The database and metadata are stored in the hidden `.git/` directory.  You can treat this as a black box, and your only interaction with the database and metadata should be through via `git` commands.  While some of the files in `.git/` are human-readible, they should *never* be manually edited, since manual edits can cause an inconsistent state.

In [6]:
ls -la

total 16
drwxr-xr-x   8 rrahaman6  staff   256 Apr 11 11:34 .
drwxr-xr-x  49 rrahaman6  staff  1568 Apr 11 10:51 ..
drwxr-xr-x  12 rrahaman6  staff   384 Apr 11 11:13 .git
drwxr-xr-x   3 rrahaman6  staff    96 Apr 11 11:19 .ipynb_checkpoints
-rw-r--r--   1 rrahaman6  staff    81 Apr 11 10:37 README.md
drwxr-xr-x   4 rrahaman6  staff   128 Apr 11 11:27 img
-rw-r--r--   1 rrahaman6  staff  2729 Apr 11 11:33 oss-module-01-git.ipynb
drwxr-xr-x   2 rrahaman6  staff    64 Apr 11 11:19 src


## Tracked vs. Untracked Files

Git does not automatically add files from the working tree to the database.  Instead, the user must explicitly specify which files are stored in the database (**tracked**) and which are not (**untracked**).

In this repo, the *tracked* files are: the README, Jupyter notebooks, `img/` directory, and `src/` directory.  If you edit and save this notebook, your working tree will have an *untracked* directory `.ipynb_checkpoints/` that are save files for Jupyter.  

It's important to consider which files to *not* track.  In a Python project, you should not track byte code (`*.pyc` and `*.pyo` in `__pycache__` directories) since they will be automatically regenerated whenever someone else runs it with a different Python version.  Tracking them will just take up unnecessary space in the database.  

# Section 4: A Git Workflow from Start to Finish

## `git init`: Creating a local repo

The `git init` command creates a new repo in the current directory.  Below, we create a new directory and initialize an repo inside it.

In [10]:
mkdir ~/git_workflow
cd ~/git_workflow
git init

Initialized empty Git repository in /Users/rrahaman6/git_workflow/.git/


You can see the `.git/` directory was created for the database.  

In [12]:
ls -lha

total 0
drwxr-xr-x    3 rrahaman6  staff    96B Apr 11 14:16 .
drwxr-xr-x+ 191 rrahaman6  staff   6.0K Apr 11 14:16 ..
drwxr-xr-x    9 rrahaman6  staff   288B Apr 11 14:16 .git


## `git help`: Getting help

`git help <command>` will show a manual page for the given command.  They describe all the available options with examples at the bottom.

In [13]:
git help init

GIT-INIT(1)                       Git Manual                       GIT-INIT(1)

NAME
       git-init - Create an empty Git repository or reinitialize an existing
       one

SYNOPSIS
       git init [-q | --quiet] [--bare] [--template=<template_directory>]
                 [--separate-git-dir <git dir>] [--object-format=<format>]
                 [-b <branch-name> | --initial-branch=<branch-name>]
                 [--shared[=<permissions>]] [directory]


DESCRIPTION
       This command creates an empty Git repository - basically a .git
       directory with subdirectories for objects, refs/heads, refs/tags, and
       template files. An initial branch without any commits will be created
       (see the --initial-branch option below for its name).

       If the $GIT_DIR environment variable is set then it specifies a path to
       use instead of ./.git for the base of the repository.

       If the object storage directory is specified via the
       $GIT_OBJECT_DIRECTORY environment 

## `git status`: Show the state of the working tree

`git status` shows the state of tracked files, untracked files, and the database. Right now, the working tree is empty, and the database is empty, so there is not much to report.  

We already have a **branch** called `main`, which was created when the repo was initialized.  We will cover branches in detail soon.  For now, you can think of them like branches of a family tree that describe your project's history.  

In [14]:
git status

On branch main

No commits yet

nothing to commit (create/copy files and use "git add" to track)
